<div align='center' ><font size='70'>Associative Container</font></div>

## 关联式容器

* **map / set**
    <img align="right" src="./src/rb_tree.png" width="65%" border="0">
    * 有序 **key**必须可序
    * 红黑树 [知乎](https://zhuanlan.zhihu.com/p/273829162)

* **unordered_map / unordered_set**
    * 无序 **key**必须可哈希
    * 哈希表

* **map**和**set**区别？

* 集合元素可以重复吗？**multiset / multimap**


## [**set**](../../224758/Source/tinySTL/src/stl_set.h)

* 内含一个**rb_tree**

* **key**就是**value**

* **set**的**value**不可修改 **const_iterator**

* 大部分接口**rb_tree**已提供

```c++
    template<typename Key, typename Compare=less<Key>,typename Alloc=alloc>
class set {
public:
	typedef Key key_type;
	typedef Key value_type;
	typedef Compare key_compare;
	typedef Compare value_compare;

private:
	typedef RB_Tree<key_type, value_type, Identity<value_type>, key_compare, Alloc> RB_type;
	RB_type m_rbt;
public:
	typedef typename RB_type::const_pointer pointer;
	typedef typename RB_type::const_pointer const_pointer;
	typedef typename RB_type::reference reference;
	typedef typename RB_type::const_reference const_reference;
	typedef typename RB_type::const_iterator iterator;
	typedef typename RB_type::const_iterator const_iterator;
    // ...
};
```

## [**map**](../../224758/Source/tinySTL/src/stl_map.h)

* 内含一个**rb_tree**

* **key**和**value**不同

* **map**的**value**可修改 **iterator**

* **value_type**是**pair**

```c++
template<typename Key, typename Tp, typename Compare=less<Key>,typename Alloc=alloc>
class map {
// ...
public:
	// value_compare
	class value_compare :public binary_function<value_type, value_type, bool> {
		friend class map<Key, Tp, Compare, Alloc>;
	protected:
		Compare cmp;
		value_compare(Compare c) :cmp(c) {}
	public:
		inline bool operator()(const value_type& x, const value_type& y)const {
			return cmp(x.first, y.first);
		}
	};
public:
	typedef Key key_type;
	typedef Tp data_type;
	typedef pair<const Key, Tp> value_type;
    // ...
private:
	typedef RB_Tree<key_type, value_type, Select1st<value_type>, key_compare, Alloc> RB_type;
	RB_type m_rbt;
// ...
};
```

<div align='center' ><font size='70'>hashtable</font></div>

<img align="right" src="./src/hashtable.png" width="40%" border="0">

## [**hashtable**](../src/stl_hashtable.h)
* **RB_Tree** 搜索、插入、删除 *O(logn)*

* **hashtable** 搜索、插入、删除 *O(1)*

* 散列函数是关键 **vs.** [**Cryptographic hash function**](https://en.wikipedia.org/wiki/Cryptographic_hash_function)

* **hashtable** 采用**开链法**解决冲突

* **iterator** 类型 **forward_iterator**

* 内含一个**vector** 如何减少**碰撞**

## **hashtable** 实现

* [**质数**](https://medium.com/swlh/why-should-the-length-of-your-hash-table-be-a-prime-number-760ec65a75d1)作为**buckets**的个数

* 预制**28**个质数 **53，97，193，389，769，1543，3079，6151，...**

* [测试](./code/hashtable.cpp)环境
    * **Windows 11 Visual Studio 2022 17.3.6**
    * **WSL 2 Ubuntu 18.04 LTS, GCC 7.5.0** <font color=red>I/O非常慢</font>
<img align="center" src="./src/hashtbale.svg" width="75%" border="0">

## **hashtable** 

* **hash function**

* 字符串 **hash function**

```c++
template<typename Key> hash{ };
inline size_t __stl_hash_string(const char* s) {
    unsigned long h = 0;
    for (; *s; ++s)
        h = 5 * h + *s;
    return size_t(h);
}
// other fundamental type
```
* 重载**operator()**，设计**hash function**

* unordered_map<**vector<int>**, int> 怎么办？
    * [**good hash function for vector**](https://stackoverflow.com/questions/20511347/a-good-hash-function-for-a-vector)
    * [**LeetCode 2352 相等行列对**](https://leetcode.cn/problems/equal-row-and-column-pairs/)


## [**hash_set**](../src/stl_hash_set.h)

* **C++11**之后的头文件 **unordered_set**

* **key就是value**

* 大部分接口**hashtable**已提供

* 迭代器类型 **forward_iterator**，不可修改

```c++
template<typename Val, typename HashFunc=hash<Val>,
	     typename EqualKey=equal_to<Val>,typename Alloc=alloc>
class unordered_set {
private:
	typedef hashtable<Val, Val, HashFunc, Identity<Val>, EqualKey, Alloc>Ht;
	Ht m_ht;

public:
	typedef typename Ht::key_type key_type;
	typedef typename Ht::value_type value_type;
	typedef typename Ht::const_pointer const_pointer;
	typedef typename Ht::const_pointer pointer;
	typedef typename Ht::const_reference reference;
	typedef typename Ht::const_reference const_reference;
	typedef typename Ht::const_iterator iterator;
	typedef typename Ht::const_iterator const_iterator;
    // ...
	hasher hash_func()const { return m_ht.hash_fun(); }
	key_equal key_eq()const { return m_ht.key_eq(); }
// ...
};
```

## [**hash_map**](../src/stl_hash_map.h)

* **C++11**之后的头文件 **unordered_map**

*  **key**和**value**不同

* **value**可修改

* **value_type**是**pair**

```c++
template<typename Key, typename Tp, typename HashFun = hash<Key>,
	typename EqualKey = equal_to<Key>,
	typename Alloc = alloc>
class unordered_map {

private:
	typedef hashtable<pair<const Key, Tp>, Key, HashFun, Select1st<pair<const Key, Tp>>, EqualKey, Alloc> Ht;
	Ht m_ht;

public:
	typedef typename Ht::key_type key_type;
	typedef Tp data_type;
	typedef Tp mapped_type;
	typedef typename Ht::value_type value_type;
	typedef typename Ht::hasher hasher;
	typedef typename Ht::key_equal key_equal;
    // ...
	typedef typename Ht::iterator iterator;
	typedef typename Ht::const_iterator const_iterator;

	inline hasher hash_func()const { return m_ht.hash_fun(); }
	inline key_equal key_eq()const { return m_ht.key_eq(); }
// ...
};
```

<div align='center' ><font size='70'>End</font></div>